## Second heuristic - Preliminary implementation

### Description

If there is a deposit and a withdraw transaction with **unique** gas prices (e.g., 3.1415926 Gwei), then we consider the deposit and the withdraw transactions linked. The corresponding deposit transaction can be removed from any other withdraw transaction’s anonymity set.

In [2]:
using DataFrames
using CSV
using ProgressBars

In [3]:
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)
deposit_transactions_df = CSV.read("../data/lighter_complete_deposit_txs.csv", DataFrame);

In [4]:
ENV["COLUMNS"]=10000
ENV["LINES"]=10;

In [25]:
withdraw_transactions_df = CSV.read("../data/lighter_complete_withdraw_txs.csv", DataFrame)

,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address,recipient_address
,Int64,Int64,String,Int64,Int64,String,String,Int64,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String,Int64,String,Float64?,Float64?,Float64?,Int64,String,String
1,0,7,0x3dc8617176f45c0f33a95589c202b22f9ba8aa7067bb92887517e20b237907e3,789,196,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0xa160cdab225685da1d56aa342ad8841c3b53f291,0,386384,121000000000,10382378,336384,missing,missing,1,2020-10-05 07:57:28 UTC,10994498,0xa96abb653dd6d4e3cedd6bd65fd0a7b034e4c9527be91cb80ee62d790714ab49,missing,missing,missing,121000000000,0xa160cdab225685da1d56aa342ad8841c3b53f291,0xd843ce0f9da3bd30537b9850711ab8089e06b4cf
2,1,8,0x3199144ba98021e523c5ec1e4f4dcf64446d69b825da7251d2d2a0a794038d0d,9,190,0xb77562124be8ac967cf7fc24573fe252aa39d95d,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13000000000,7303106,339456,missing,missing,1,2020-02-11 19:10:54 UTC,9463504,0x1e1dc9cec82d1dcd0e68e60968d180408007e148ac065261de232dc502dfb756,missing,missing,missing,13000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0xb77562124be8ac967cf7fc24573fe252aa39d95d
3,2,9,0x95a46a1305ddc8b7e01d82c7d6766fce4713c085929f70c9a24e90f927672699,2,81,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,439456,13200000000,3211601,339456,missing,missing,1,2020-02-16 16:05:14 UTC,9495139,0x1f0562c98f40f0689ed8bac422b4714289bd452d9290928b30f7defe0943381f,missing,missing,missing,13200000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0x0e54db73f82bd9fde34ebce53ea83bd197e9044c
4,3,13,0x172a0a1618aa6c1f9f27a370236d8c91156fd6192ad11b91faa77bb852599b76,2024,189,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,384581,40000000000,11795370,343296,missing,missing,1,2020-11-09 17:08:00 UTC,11224582,0xe0ae8902495001fc28025238a74703c1b3d58282aaeef5fd5ca3fe9cb2d273ff,missing,missing,missing,40000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x42a9cf901e889ca8a685394854c27898dbff86b0
5,4,15,0x30becee80e63a039102df3b1243e5d314534abfcf3ed0fc4eadb2dc5b1c45ab7,2350,127,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,15020000842,9771737,354304,missing,missing,1,2020-11-29 01:18:57 UTC,11350558,0x4d8f724d2bdffa546cef74d21c6c43968b566b67318e2f737a7c6b26b8781e56,missing,missing,missing,15020000842,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0xc9fb4b16800e61076326145e1066e876cf9f7a7b
6,5,16,0x5ef2e6d75efa800f009a84b27ae22609597edee8df8ee8699c004af233cf5855,999,120,0x2d50dbcc960bec854227f51f8074e1da28bd4a37,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0,404304,72000000000,8783549,354304,missing,missing,1,2020-10-11 18:09:09 UTC,11035787,0xb18729068418804f6c247b13e92236035f8303c43828c59d70b279166ca4f67a,missing,missing,missing,72000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,0x9453663da4123648bd00cb5600b97a4314ad0058
7,6,17,0x59d6d259118b41170550e5662c35b20c1b8ba795e420f6ce7a74772a9a834492,550,86,0x41a28335c5075c81502a97cebad597f28728a815,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,404304,26000000000,4476696,354304,missing,missing,1,2020-05-26 22:55:55 UTC,10144176,0xd22e4252818d56d9a7b4bb6c7488842877db42771a497f175ba78cd7a1aa00c1,missing,missing,missing,26000000000,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0xd01ad76a2337354dd458fcbb6e507e001f06b77d
8,7,42,0x805b2ae79b5267a23e6c79d3324d43dae0f64e81f77f166c09dca9ff46cfc955,260,128,0xefab18983029d2ba840e34698efb67fdf8120711,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0,383057,5050000000,7284273,333057,missing,missing,1,2020-02-04 13:43:11 UTC,9416438,0x644ea6d8f078b89696758c59f74f93f0f226838593a79bcb9980f5815e023cbb,missing,missing,missing,5050000000,0x12d66f87a04a9e22074

### Function summary: filter_by_unique_gas_price

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions.

In [20]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price.

function filter_by_unique_gas_price(transactions_df)
    unique_gas_prices = filter(row -> row.count==1, combine(groupby(transactions_df, :gas_price), nrow => :count))[!, "gas_price"]
    filter(row -> row.gas_price ∈ unique_gas_prices, transactions_df)
end

filter_by_unique_gas_price (generic function with 1 method)

### Function summary: filter_by_unique_gas_price_by_pool

Filters a transaction DataFrame, leaving only the transactions (rows) that have unique gas_price within all transactions of the same pool.

In [151]:
# Filters a transaction DataFrame, leaving only the rows that have unique gas_price.

function filter_by_unique_gas_price_by_pool(transactions_df)
    unique_gas_prices_per_pool = filter(row -> row.count == 1, combine(groupby(transactions_df, [:gas_price, :tornado_cash_address]), nrow => :count))[!, [:gas_price, :tornado_cash_address]]
    tuple_set = Set([(row.gas_price, row.tornado_cash_address) for row in eachrow(unique_gas_prices_per_pool)])
    
    filter(row -> (row.gas_price, row.tornado_cash_address) ∈ tuple_set, transactions_df)
end

filter_by_unique_gas_price_by_pool (generic function with 1 method)

In [158]:
@time filter_by_unique_gas_price_by_pool(deposit_transactions_df)

  0.079204 seconds (773.90 k allocations: 18.871 MiB)


,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,Int64,String,Int64,Int64,String,String,Int128,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String31,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,0,0,0xcf97c470a56d96625c7240d3004ae2abd9141d7ffc4383ab6f29a181e3562e8b,4,10,0xb050dec5a9010f8b77a3962369b7bc737d3ed4a5,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d,0,1200000,56000000000,1677906,1048832,missing,missing,1,2020-11-02 17:47:30 UTC,11179130,0x21d86cba454fea4f7e43c68763d4cffec101b614554635a3f15d538049463d4f,missing,missing,missing,56000000000,0x4736dcf1b7a3d580672cce6e7c65cd5cc9cfba9d
2,2,10,0x7baf0a76f35c1dece97fff883aa7174454bed460b1bade05844080017170fc6c,240,171,0x8c4c44fd06f7f98f08bf6a9ca156cec9ee1f31f8,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144,0,800000,105000000000,10723996,800000,missing,missing,0,2021-01-06 19:04:40 UTC,11602841,0x7c5f21ea2a92f5182ce8648f152b6fb3b4379096309dc2ba232ac663c4a0d1d7,missing,missing,missing,105000000000,0xfd8610d20aa15b7b2e3be39b396a1bc3516c7144
3,4,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
4,8,130,0xba7d56fea776705a937d912674cc56cf3ea71485c8fb6995d1c2ec00d44645fc,2,69,0x3a456bc9083bfe147719504aee8f296eb7355ee1,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0,1200000,100000000000,4634190,992258,missing,missing,1,2020-08-15 11:34:23 UTC,10664413,0x4075362cd93950767b2e5c3cd6765810b2973a2dd073186ed75141973fb1ae84,missing,missing,missing,100000000000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
5,10,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0,1200000,71302125000,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
6,12,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1000000000000000000,275000,595000000000,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
7,17,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10000000000000000000,1200000,5231046931,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
8,67,1632,0x923c87764f1be67385a3ee07376b9c4a640c7a6402525337389654a35b8b57cd,205,13,0x67865c3eaf2e6a745a6b85cee169c7b74abe97d4,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10000000000000000000,1200000,6500000000,4612395,978691,missing,missing,1,2020-02-28 03:00:12 UTC,9569564,0xd6b7228ba576816050a47c4c7f4c21b96d5fc384f3a68a67d5866cc59843a648,missing,missing,missing,6500000000,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
9,125,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,57151624549,9371665,978

In [21]:
unique_gas_price_deposits = filter_by_unique_gas_price(deposit_transactions_df)
first(unique_gas_price_deposits, 5)

,Column1,Unnamed: 0,hash,nonce,transaction_index,from_address,to_address,value,gas,gas_price,receipt_cumulative_gas_used,receipt_gas_used,receipt_contract_address,receipt_root,receipt_status,block_timestamp,block_number,block_hash,max_fee_per_gas,max_priority_fee_per_gas,transaction_type,receipt_effective_gas_price,tornado_cash_address
,Int64,Int64,String,Int64,Int64,String,String,Int128,Int64,Int64,Int64,Int64,Missing,Missing,Int64,String31,Int64,String,Float64?,Float64?,Float64?,Int64,String
1,4,12,0xbd83053f8afa7777f54a4aca6b8e112fa31b888922dc5b9a9a65eb66e9a6996f,7,63,0x6c6e4816ecfa4481472ff88f32a3e00f2eaa95a1,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,800000,30838446643,6222489,800000,missing,missing,0,2020-05-27 03:30:44 UTC,10145408,0x837b3482443f027f6f045644bf002243f72304686015a2d6265676b2a2fc630b,missing,missing,missing,30838446643,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc
2,10,132,0x6c416af65ea3a4bc096663c94f5b1fb0cba91607f61703657094f1f5441a3a12,3,51,0x27972d10f153099b3649ea8546a11d91315455e5,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f,0,1200000,71302125000,4108630,992258,missing,missing,1,2020-09-26 08:10:08 UTC,10937092,0x32f0f0fd04d3af8210d2eb956fdec21e09ebff5a55209c0a2c559bb1c034b158,missing,missing,missing,71302125000,0x0836222f2b2b24a3f36f98668ed8f0b38d1a872f
3,12,1549,0x830dbd534d13cd43cb078b7cad8a9c5137bb19aa8bf38e0c3b0e222b688d8340,0,63,0x43eefeb3db479e7b22e015572f38b6af633a43ff,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936,1000000000000000000,275000,595000000000,4390453,274947,missing,missing,0,2020-09-17 20:50:21 UTC,10881994,0x5270afd78906cc7264620b2c6fbf8c9221cf8053c8111edc0d7457966340dd81,missing,missing,missing,595000000000,0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936
4,17,1582,0x56837665efa6b3ed43685c79744f8b3c575090f7d076f7187c2ae3ace5aea0c9,8,41,0x0fcdae4a4aeb0497f39a209009dc200a6466b7bd,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10000000000000000000,1200000,5231046931,5058769,978691,missing,missing,1,2020-01-24 05:54:15 UTC,9342713,0x2263281e65fb7d50125309a90864131650ca560ccd51c3e8e2847fbd2a18ee9f,missing,missing,missing,5231046931,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf
5,125,1690,0x038303d14327a2437bd789a70c0f7daa95b3ab89b46af0418fd0f677bdf2766c,82,163,0x3eb24c61590e6cd6fc26cf4fb938316206147859,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,100000000000000000,1200000,57151624549,9371665,978691,missing,missing,1,2020-10-16 09:18:01 UTC,11066064,0xfa752cf56aaec471375e2996736d20444b146199e2c3b84851cb16c65695cd3f,missing,missing,missing,57151624549,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc


### Function summary: same_gas_price_heuristic

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction is found.
* $(false, nothing)$ when such a deposit is not found.

In [26]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.

function same_gas_price_heuristic(withdrawal_transaction, unique_gas_price_deposit_dataframe)
    for row in eachrow(unique_gas_price_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic (generic function with 1 method)

### Function summary: same_gas_price_heuristic_by_pool

This function receives a particular withdraw transaction and a DataFrame with the unique gas price deposits by pool.

It returns a tuple:
* $(true, deposit$ $hash)$ when a deposit transaction with the same gas price as the withdrawal transaction and from the same TCash pool is found.
* $(false, nothing)$ when such a deposit is not found.

In [35]:
# Given a withdrawal transaction and a DataFrame with unique gas_price deposit transactions, checks
# if there is a deposit transaction with the same gas_price as the withdrawal transaction.

function same_gas_price_heuristic_by_pool(withdrawal_transaction, unique_gas_price_by_pool_deposit_dataframe)
    for row in eachrow(unique_gas_price_by_pool_deposit_dataframe)
        if (withdrawal_transaction.gas_price == row.gas_price) &&
            (withdrawal_transaction.block_timestamp > row.block_timestamp) &&
            (withdrawal_transaction.tornado_cash_address == row.tornado_cash_address)
                return (true, row.hash)
        end
    end
    (false, nothing)
end

same_gas_price_heuristic_by_pool (generic function with 1 method)

In [38]:
same_gas_price_heuristic(withdraw_transactions_df[1,:], unique_gas_price_deposits)

(false, nothing)

### Function summary: apply_same_gas_price_heuristic

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [160]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.

function apply_same_gas_price_heuristic(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price(deposit_dataframe)
    
    withdrawal_to_deposit = Dict()
    
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe), printing_delay=3)
        same_gas_deposit_hash = same_gas_price_heuristic(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic (generic function with 1 method)

### Function summary: apply_same_gas_price_heuristic_by_pool

Applies the heuristic to all the withdraw_transactions DataFrame. Returns a dicionary mapping linked withdrawal and deposit transaction hashes.

In [163]:
# Applies the function to detect same unique gas_prices to all the withdraw_transactions data.
# Returns a list of tuples, each tuple with the deposit transaction hash in the first index and
# the withdrawal transaction hash in the second index.

function apply_same_gas_price_heuristic_by_pool(deposit_dataframe, withdraw_dataframe)
    unique_gas_price_deposits = filter_by_unique_gas_price_by_pool(deposit_dataframe)
    
    withdrawal_to_deposit = Dict()
    
    for withdraw_row in ProgressBar(eachrow(withdraw_dataframe), printing_delay=3)
        same_gas_deposit_hash = same_gas_price_heuristic_by_pool(withdraw_row, unique_gas_price_deposits)
        if same_gas_deposit_hash[1]
            withdrawal_to_deposit[withdraw_row.hash] = same_gas_deposit_hash[2]
        end
    end
    withdrawal_to_deposit
end 

apply_same_gas_price_heuristic_by_pool (generic function with 1 method)

In [40]:
@time linked_transactions = apply_same_gas_price_heuristic(deposit_transactions_df, withdraw_transactions_df);

0.0%┣                                        ┫ 0/83.8k [00:00<-1:-9:-49, -0s/it]
0.0%┣                                        ┫ 1/83.8k [00:00<Inf:Inf, InfGs/it]
0.0%┣                                          ┫ 34/83.8k [00:00<04:21, 321it/s]
0.1%┣                                          ┫ 68/83.8k [00:00<03:12, 435it/s]
0.1%┣                                          ┫ 94/83.8k [00:00<03:04, 454it/s]
0.2%┣                                         ┫ 127/83.8k [00:00<02:50, 493it/s]
0.2%┣                                         ┫ 153/83.8k [00:00<02:49, 495it/s]
0.2%┣                                         ┫ 186/83.8k [00:00<02:42, 516it/s]
0.2%┣                                         ┫ 204/83.8k [00:00<02:59, 467it/s]
0.3%┣▏                                        ┫ 238/83.8k [00:00<02:51, 488it/s]
0.3%┣▏                                        ┫ 263/83.8k [00:01<02:51, 487it/s]
0.4%┣▏                                        ┫ 296/83.8k [00:01<02:47, 501it/s]
0.4%┣▏                      

3.1%┣█▎                                      ┫ 2.6k/83.8k [00:05<02:47, 486it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:05<02:47, 486it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<02:46, 487it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<02:47, 486it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<02:48, 484it/s]
3.3%┣█▎                                      ┫ 2.7k/83.8k [00:06<02:48, 483it/s]
3.3%┣█▎                                      ┫ 2.8k/83.8k [00:06<02:47, 485it/s]
3.3%┣█▍                                      ┫ 2.8k/83.8k [00:06<02:47, 485it/s]
3.4%┣█▍                                      ┫ 2.8k/83.8k [00:06<02:47, 486it/s]
3.4%┣█▍                                      ┫ 2.9k/83.8k [00:06<02:47, 486it/s]
3.4%┣█▍                                      ┫ 2.9k/83.8k [00:06<02:46, 487it/s]
3.5%┣█▍                                      ┫ 2.9k/83.8k [00:06<02:47, 484it/s]
3.5%┣█▍                     

6.2%┣██▌                                     ┫ 5.2k/83.8k [00:11<02:42, 486it/s]
6.2%┣██▌                                     ┫ 5.2k/83.8k [00:11<02:42, 486it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:11<02:41, 487it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:11<02:41, 487it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:11<02:41, 487it/s]
6.4%┣██▌                                     ┫ 5.3k/83.8k [00:11<02:41, 487it/s]
6.4%┣██▋                                     ┫ 5.4k/83.8k [00:11<02:41, 488it/s]
6.4%┣██▋                                     ┫ 5.4k/83.8k [00:11<02:41, 486it/s]
6.4%┣██▋                                     ┫ 5.4k/83.8k [00:11<02:41, 486it/s]
6.5%┣██▋                                     ┫ 5.4k/83.8k [00:11<02:41, 486it/s]
6.5%┣██▋                                     ┫ 5.4k/83.8k [00:11<02:41, 486it/s]
6.5%┣██▋                                     ┫ 5.5k/83.8k [00:11<02:41, 486it/s]
6.6%┣██▋                    

9.3%┣███▊                                    ┫ 7.8k/83.8k [00:16<02:36, 488it/s]
9.4%┣███▊                                    ┫ 7.8k/83.8k [00:16<02:36, 487it/s]
9.4%┣███▊                                    ┫ 7.9k/83.8k [00:16<02:36, 487it/s]
9.4%┣███▊                                    ┫ 7.9k/83.8k [00:16<02:36, 487it/s]
9.5%┣███▉                                    ┫ 7.9k/83.8k [00:16<02:36, 488it/s]
9.5%┣███▉                                    ┫ 8.0k/83.8k [00:16<02:36, 488it/s]
9.5%┣███▉                                    ┫ 8.0k/83.8k [00:16<02:35, 488it/s]
9.6%┣███▉                                    ┫ 8.0k/83.8k [00:16<02:35, 488it/s]
9.6%┣███▉                                    ┫ 8.0k/83.8k [00:16<02:36, 487it/s]
9.6%┣███▉                                    ┫ 8.0k/83.8k [00:17<02:35, 487it/s]
9.6%┣███▉                                    ┫ 8.1k/83.8k [00:17<02:35, 488it/s]
9.7%┣███▉                                    ┫ 8.1k/83.8k [00:17<02:35, 487it/s]
9.7%┣███▉                   

12.4%┣████▊                                 ┫ 10.4k/83.8k [00:21<02:31, 487it/s]
12.4%┣████▊                                 ┫ 10.4k/83.8k [00:21<02:30, 487it/s]
12.5%┣████▊                                 ┫ 10.5k/83.8k [00:21<02:30, 488it/s]
12.5%┣████▊                                 ┫ 10.5k/83.8k [00:21<02:30, 488it/s]
12.5%┣████▊                                 ┫ 10.5k/83.8k [00:22<02:30, 487it/s]
12.5%┣████▊                                 ┫ 10.5k/83.8k [00:22<02:30, 487it/s]
12.6%┣████▉                                 ┫ 10.5k/83.8k [00:22<02:30, 487it/s]
12.6%┣████▉                                 ┫ 10.6k/83.8k [00:22<02:30, 487it/s]
12.7%┣████▉                                 ┫ 10.6k/83.8k [00:22<02:30, 488it/s]
12.7%┣████▉                                 ┫ 10.6k/83.8k [00:22<02:30, 487it/s]
12.7%┣████▉                                 ┫ 10.6k/83.8k [00:22<02:30, 487it/s]
12.7%┣████▉                                 ┫ 10.7k/83.8k [00:22<02:30, 486it/s]
12.8%┣████▉                 

...

93.0%┣███████████████████████████████████▎  ┫ 77.9k/83.8k [02:44<00:12, 476it/s]
93.0%┣███████████████████████████████████▍  ┫ 77.9k/83.8k [02:44<00:12, 476it/s]
93.0%┣███████████████████████████████████▍  ┫ 78.0k/83.8k [02:44<00:12, 476it/s]
93.1%┣███████████████████████████████████▍  ┫ 78.0k/83.8k [02:44<00:12, 476it/s]
93.1%┣███████████████████████████████████▍  ┫ 78.0k/83.8k [02:44<00:12, 476it/s]
93.1%┣███████████████████████████████████▍  ┫ 78.0k/83.8k [02:44<00:12, 476it/s]
93.2%┣███████████████████████████████████▍  ┫ 78.1k/83.8k [02:44<00:12, 476it/s]
93.2%┣███████████████████████████████████▍  ┫ 78.1k/83.8k [02:44<00:12, 476it/s]
93.2%┣███████████████████████████████████▍  ┫ 78.1k/83.8k [02:44<00:12, 476it/s]
93.3%┣███████████████████████████████████▍  ┫ 78.1k/83.8k [02:44<00:12, 476it/s]
93.3%┣███████████████████████████████████▌  ┫ 78.2k/83.8k [02:44<00:12, 476it/s]
93.3%┣███████████████████████████████████▌  ┫ 78.2k/83.8k [02:44<00:12, 476it/s]
93.4%┣██████████████████████

95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [02:49<00:08, 473it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [02:49<00:08, 473it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [02:49<00:08, 473it/s]
95.6%┣████████████████████████████████████▍ ┫ 80.1k/83.8k [02:49<00:08, 473it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:49<00:08, 473it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:49<00:08, 473it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:49<00:08, 473it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:50<00:08, 473it/s]
95.7%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:50<00:08, 473it/s]
95.8%┣████████████████████████████████████▍ ┫ 80.2k/83.8k [02:50<00:07, 473it/s]
95.8%┣████████████████████████████████████▍ ┫ 80.3k/83.8k [02:50<00:07, 473it/s]
95.8%┣████████████████████████████████████▍ ┫ 80.3k/83.8k [02:50<00:07, 473it/s]
95.9%┣██████████████████████

98.4%┣█████████████████████████████████████▍┫ 82.5k/83.8k [02:55<00:03, 472it/s]
98.5%┣█████████████████████████████████████▍┫ 82.5k/83.8k [02:55<00:03, 472it/s]
98.5%┣█████████████████████████████████████▍┫ 82.5k/83.8k [02:55<00:03, 472it/s]
98.5%┣█████████████████████████████████████▍┫ 82.5k/83.8k [02:55<00:03, 472it/s]
98.5%┣█████████████████████████████████████▍┫ 82.6k/83.8k [02:55<00:03, 472it/s]
98.6%┣█████████████████████████████████████▌┫ 82.6k/83.8k [02:55<00:03, 472it/s]
98.6%┣█████████████████████████████████████▌┫ 82.6k/83.8k [02:55<00:02, 472it/s]
98.6%┣█████████████████████████████████████▌┫ 82.6k/83.8k [02:55<00:02, 472it/s]
98.7%┣█████████████████████████████████████▌┫ 82.7k/83.8k [02:55<00:02, 472it/s]
98.7%┣█████████████████████████████████████▌┫ 82.7k/83.8k [02:55<00:02, 472it/s]
98.7%┣█████████████████████████████████████▌┫ 82.7k/83.8k [02:55<00:02, 472it/s]
98.7%┣█████████████████████████████████████▌┫ 82.7k/83.8k [02:55<00:02, 472it/s]
98.8%┣██████████████████████

178.244300 seconds (4.88 G allocations: 72.697 GiB, 9.63% gc time, 0.02% compilation time)


100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [02:58<00:00, 471it/s]
100.0%┣█████████████████████████████████████┫ 83.8k/83.8k [02:58<00:00, 471it/s]


In [42]:
# A DataFrame with the linked transactions.
linked_transactions_df = DataFrame("withdraw_hash"=> collect(keys(linked_transactions)), "deposit_hash"=> collect(values(linked_transactions)))

,withdraw_hash,deposit_hash
,Any,Any
1,0xf58ae971b970c6b2629627988f1d5f88cd23b763addd8b1c6b126e036dce2cea,0x00b6cc296beb65377ddb94e2b135f78fa6f26ba3a1d770491aeae0c774b7042f
2,0x11ec2acf23f18cecd5f892ac4f00d887d58b3b96fede4938e451b572ea5ceaba,0x2d714ec34a44bd097cc4fdbf7cf643762a6bcf6fee6dc32e18c2853e8d0309e4
3,0x8fea2a0cefa4b760512b65aac445c2f4e5e668753ce4da00e6f98d34ba38b280,0x785c4a2b73b892791d609e8f622aa7e528579094bc93ec2c4ac88d19a333dee0
4,0x8ee98c3c23a18acca2f48dc2c00d583dba9f46ab5ce3d44e77203a7b8e0b1990,0x785c4a2b73b892791d609e8f622aa7e528579094bc93ec2c4ac88d19a333dee0
5,0x73a88cf8ce24fee66ec64dc9d6c77dc6067688ef4fa12383dbf3d86e1e380e82,0xfb64bc5259d9cf5b230ed63a376724ba3ec14e3cfe40d6d3ddbcfa5fec3f1393
6,0xb9db494857677a53e3d0b56418f3d3070461b77198d4d462f6dd1291c3157e16,0xbbdd6a96107d1e7e86737e04220f68eab068140f30f778711dd7a7dc6cf4c310
7,0x96eb0f9464ede13f3d0cc7b1c5c185f6d86957e255d15f542b2251dc52ddf867,0x21670c83c88f3c281d804c93b70b9f65a2d0b8ece49f80015c12895db8dcfed0
8,0x6c0666bcb78ee851fa92f0c89c67c6e5f231fe4ccadbf97798660f1034a44b66,0xfa1414e82c8acea9a304d7831ff1f8c7f2a67c81e5e059711e0b3adb56ca9a50
9,0xef11465a6ff7ae3d782d258aee6f3207860a20642b1fe7b3a489cf0f39a8788d,0xe00273af3cda8af3f230692ae0a5122a51b1b3ebcf9cb799049ec5a78c9ddc6e


In [154]:
@time linked_transactions_by_pool = apply_same_gas_price_heuristic_by_pool(deposit_transactions_df, withdraw_transactions_df);

0.0%┣                                        ┫ 0/83.8k [00:00<-1:-9:-49, -0s/it]
0.0%┣                                        ┫ 1/83.8k [00:00<Inf:Inf, InfGs/it]
0.0%┣                                          ┫ 15/83.8k [00:00<12:22, 113it/s]
0.1%┣                                          ┫ 43/83.8k [00:00<05:51, 238it/s]
0.1%┣                                          ┫ 69/83.8k [00:00<04:41, 297it/s]
0.1%┣                                          ┫ 88/83.8k [00:00<04:31, 309it/s]
0.1%┣                                         ┫ 103/83.8k [00:00<04:34, 306it/s]
0.1%┣                                         ┫ 118/83.8k [00:00<04:36, 304it/s]
0.2%┣                                         ┫ 136/83.8k [00:00<04:30, 310it/s]
0.2%┣                                         ┫ 152/83.8k [00:00<04:30, 309it/s]
0.2%┣                                         ┫ 168/83.8k [00:01<04:30, 309it/s]
0.2%┣                                         ┫ 176/83.8k [00:01<04:42, 296it/s]
0.2%┣                       

3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:09, 430it/s]
3.0%┣█▏                                      ┫ 2.5k/83.8k [00:06<03:09, 429it/s]
3.1%┣█▏                                      ┫ 2.6k/83.8k [00:06<03:10, 428it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:10, 428it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:10, 428it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:09, 429it/s]
3.1%┣█▎                                      ┫ 2.6k/83.8k [00:06<03:10, 427it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:10, 427it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:10, 427it/s]
3.2%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:09, 428it/s]
3.3%┣█▎                                      ┫ 2.7k/83.8k [00:06<03:09, 429it/s]
3.3%┣█▎                                      ┫ 2.8k/83.8k [00:06<03:09, 429it/s]
3.3%┣█▎                     

6.0%┣██▍                                     ┫ 5.1k/83.8k [00:12<03:00, 438it/s]
6.1%┣██▍                                     ┫ 5.1k/83.8k [00:12<03:00, 438it/s]
6.1%┣██▍                                     ┫ 5.1k/83.8k [00:12<03:00, 438it/s]
6.1%┣██▌                                     ┫ 5.1k/83.8k [00:12<02:59, 438it/s]
6.1%┣██▌                                     ┫ 5.1k/83.8k [00:12<02:59, 438it/s]
6.2%┣██▌                                     ┫ 5.2k/83.8k [00:12<02:59, 438it/s]
6.2%┣██▌                                     ┫ 5.2k/83.8k [00:12<03:00, 438it/s]
6.2%┣██▌                                     ┫ 5.2k/83.8k [00:12<02:59, 438it/s]
6.2%┣██▌                                     ┫ 5.2k/83.8k [00:12<02:59, 438it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:12<02:59, 438it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:12<02:59, 439it/s]
6.3%┣██▌                                     ┫ 5.3k/83.8k [00:12<02:59, 438it/s]
6.3%┣██▌                    

9.0%┣███▋                                    ┫ 7.6k/83.8k [00:17<02:52, 442it/s]
9.1%┣███▋                                    ┫ 7.6k/83.8k [00:17<02:53, 441it/s]
9.1%┣███▋                                    ┫ 7.6k/83.8k [00:17<02:53, 442it/s]
9.1%┣███▋                                    ┫ 7.6k/83.8k [00:17<02:52, 442it/s]
9.2%┣███▋                                    ┫ 7.7k/83.8k [00:17<02:52, 442it/s]
9.2%┣███▊                                    ┫ 7.7k/83.8k [00:17<02:52, 442it/s]
9.2%┣███▊                                    ┫ 7.7k/83.8k [00:17<02:52, 442it/s]
9.2%┣███▊                                    ┫ 7.7k/83.8k [00:18<02:52, 441it/s]
9.3%┣███▊                                    ┫ 7.8k/83.8k [00:18<02:52, 441it/s]
9.3%┣███▊                                    ┫ 7.8k/83.8k [00:18<02:52, 441it/s]
9.3%┣███▊                                    ┫ 7.8k/83.8k [00:18<02:52, 442it/s]
9.3%┣███▊                                    ┫ 7.8k/83.8k [00:18<02:52, 442it/s]
9.4%┣███▊                   

12.0%┣████▋                                 ┫ 10.1k/83.8k [00:23<02:46, 445it/s]
12.0%┣████▋                                 ┫ 10.1k/83.8k [00:23<02:46, 445it/s]
12.0%┣████▋                                 ┫ 10.1k/83.8k [00:23<02:46, 445it/s]
12.1%┣████▋                                 ┫ 10.1k/83.8k [00:23<02:46, 445it/s]
12.1%┣████▋                                 ┫ 10.1k/83.8k [00:23<02:46, 445it/s]
12.1%┣████▋                                 ┫ 10.2k/83.8k [00:23<02:46, 444it/s]
12.2%┣████▋                                 ┫ 10.2k/83.8k [00:23<02:46, 445it/s]
12.2%┣████▋                                 ┫ 10.2k/83.8k [00:23<02:45, 445it/s]
12.2%┣████▋                                 ┫ 10.2k/83.8k [00:23<02:45, 445it/s]
12.2%┣████▋                                 ┫ 10.3k/83.8k [00:23<02:45, 445it/s]
12.3%┣████▋                                 ┫ 10.3k/83.8k [00:23<02:45, 445it/s]
12.3%┣████▊                                 ┫ 10.3k/83.8k [00:23<02:45, 444it/s]
12.3%┣████▊                 

...

92.0%┣███████████████████████████████████   ┫ 77.1k/83.8k [03:09<00:16, 408it/s]
92.0%┣███████████████████████████████████   ┫ 77.1k/83.8k [03:09<00:16, 408it/s]
92.1%┣███████████████████████████████████   ┫ 77.1k/83.8k [03:09<00:16, 408it/s]
92.1%┣███████████████████████████████████   ┫ 77.1k/83.8k [03:09<00:16, 408it/s]
92.1%┣███████████████████████████████████   ┫ 77.2k/83.8k [03:09<00:16, 408it/s]
92.1%┣███████████████████████████████████   ┫ 77.2k/83.8k [03:09<00:16, 408it/s]
92.2%┣███████████████████████████████████   ┫ 77.2k/83.8k [03:09<00:16, 408it/s]
92.2%┣███████████████████████████████████   ┫ 77.2k/83.8k [03:09<00:16, 408it/s]
92.2%┣███████████████████████████████████   ┫ 77.3k/83.8k [03:10<00:16, 408it/s]
92.2%┣███████████████████████████████████   ┫ 77.3k/83.8k [03:10<00:16, 408it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [03:10<00:16, 408it/s]
92.3%┣███████████████████████████████████   ┫ 77.3k/83.8k [03:10<00:16, 408it/s]
92.3%┣██████████████████████

95.4%┣████████████████████████████████████▎ ┫ 79.9k/83.8k [03:16<00:09, 408it/s]
95.4%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.5%┣████████████████████████████████████▎ ┫ 80.0k/83.8k [03:16<00:09, 408it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:16<00:09, 408it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:16<00:09, 408it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:16<00:09, 408it/s]
95.6%┣████████████████████████████████████▎ ┫ 80.1k/83.8k [03:16<00:09, 408it/s]
95.6%┣██████████████████████

97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.7%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 81.9k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:05, 404it/s]
97.8%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:04, 404it/s]
97.9%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:04, 404it/s]
97.9%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:04, 404it/s]
97.9%┣█████████████████████████████████████▏┫ 82.0k/83.8k [03:23<00:04, 404it/s]
97.9%┣██████████████████████

208.999172 seconds (5.65 G allocations: 84.178 GiB, 8.68% gc time, 0.01% compilation time)


▌┫ 82.8k/83.8k [03:26<00:03, 402it/s]
98.8%┣█████████████████████████████████████▌┫ 82.8k/83.8k [03:26<00:03, 402it/s]
98.8%┣█████████████████████████████████████▌┫ 82.8k/83.8k [03:26<00:02, 402it/s]
98.8%┣█████████████████████████████████████▋┫ 82.8k/83.8k [03:26<00:02, 402it/s]
98.9%┣█████████████████████████████████████▋┫ 82.8k/83.8k [03:26<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.8k/83.8k [03:26<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.8k/83.8k [03:26<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:26<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:26<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:27<00:02, 401it/s]
98.9%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:27<00:02, 401it/s]
99.0%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:27<00:02, 401it/s]
99.0%┣█████████████████████████████████████▋┫ 82.9k/83.8k [03:27<00:02,

In [155]:
# A DataFrame with the linked transactions by pool.
linked_transactions_by_pool_df = DataFrame("withdraw_hash"=> collect(keys(linked_transactions_by_pool)), "deposit_hash"=> collect(values(linked_transactions_by_pool)))

,withdraw_hash,deposit_hash
,Any,Any
1,0x21405e3efc65fcd5baf1b7cd08f16c82c90bab97a4fedd51d6b627d39064d11e,0x3b8be76c19fd08bdf4cd22a3165864a954c5c4c924baf646947e91227aa0c88d
2,0x1b23be3a3e3999ce712d6757730a36e9b541112342eefc485b33b1eaf5216dbc,0xa34aa895a32c1ce6001e76cc922191dc3233e8f42b8211d3be053df3dcd0f5ad
3,0x2700c7f457ec34a805b4a7f110090c9c2f33c6a647169019a9c81d3fc4b0d1a9,0x1faa62e08ab375449a979704dc886e50a841f4b938ed212514755572a0c12437
4,0x24c9bdc4737710afdbe02943bb5cfb750a15d1daef850d6f6d9d7015eac9e011,0x1a1ee8717ce269abe5aa75c7fbc952c14d5d737d87367159b793415fd44df417
5,0x11ec2acf23f18cecd5f892ac4f00d887d58b3b96fede4938e451b572ea5ceaba,0x2d714ec34a44bd097cc4fdbf7cf643762a6bcf6fee6dc32e18c2853e8d0309e4
6,0x88fde9e123ac0a9b011428ffb713c424c8aa8c5980a23fb7d8f2ba157f50d29e,0x0e1295215085d3f9e4799e8fd2a70fba45ce15dad27e284b71a503a630f5f542
7,0x572ab3201399a3cf593b01bba294358e4aa1bd9d917b0fbe7312ea78a4e7bff1,0x5b23d306c0bcc4f4b82831844e51b43fadc354665e24cc5eb512edcdf6807d7f
8,0x2ca3b5c98c952d9fb82aa83acbc3ea28b46c6df3df254605875070a256717387,0xe1d108e38ae6f563facba48f7d23194c980c4122e09609a3813119d8ef2a4440
9,0x5ec642d048fe450500d9569e1e74dcfcdbe4d03ef4dd7c10a2bbda678d18fe3c,0x64d55a7c72bd6107f8597c72765d7ac0e0c4e64217362727bed004e82cabbd1e


### Lets save into CSV File the linked transactions

In [26]:
CSV.write("../data/heuristic_2_results.csv", linked_transactions_by_pool_df)

"../data/heuristic_2_results.csv"